In [6]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../kaggle_prediction_library/') 
import preprocess
import feature_engineering
import submission
import validation
from sklearn.model_selection import train_test_split

# from hyperopt import tpe, fmin, Trials
# import hyperopt.hp as hp

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss
from sklearn.pipeline import make_pipeline



In [7]:
to_predict_mens = pd.read_csv("to_predict_mens.csv")


### Prepare data 

In [14]:
def prepare_data(to_predict_mens, season):

    to_predict_mens_train = to_predict_mens[(to_predict_mens.Season < season)].copy()
    
    to_predict_mens_test = to_predict_mens[(to_predict_mens.Season == season)].copy()
    
    return to_predict_mens_train, to_predict_mens_test
    


### Statistics Model Training

In [9]:
def train_statistics_model(to_predict_mens_train, statistics_features):

    best_params = {"C": .1}
    model = LogisticRegression(**best_params)
    pipeline = make_pipeline(StandardScaler(), model)
    statistics_model = pipeline.fit(to_predict_mens_train[statistics_features], to_predict_mens_train["Outcome"])

    return statistics_model


### Inference

In [10]:
def inference(to_predict_test, statistics_model, features):

    pred_proba = statistics_model.predict_proba(to_predict_test[features].copy())[:,1]
    to_predict_test["Pred"] = pred_proba
    mens_sub = to_predict_test[["ID", "Pred"]]
    
    return mens_sub



### Full Pipeline

In [11]:
def run(to_predict_mens, season, statistics_features):
    
    # get data
    to_predict_mens_train, to_predict_test= prepare_data(to_predict_mens, season)
    
    # train
    statistics_model = train_statistics_model(to_predict_mens_train, statistics_features)

    # inference
    sub = inference(to_predict_test, statistics_model, statistics_features) 

    return sub
    

In [16]:
statistics_features = ['t1_adj_margin', 't2_adj_margin', 't1_final_rank', 't2_final_rank', 't1_OrdinalRank', 't2_OrdinalRank']



In [17]:

all_subs = []

for year in range(2010, 2024):
    if year != 2020:
        sub = run(to_predict_mens, year, statistics_features)
        all_subs.append(sub)




In [18]:
all_subs = pd.concat(all_subs)

In [61]:
all_subs.to_csv("../submissions/all_sky_model_preds_old.csv")

#mens_sub.to_csv("../submissions/sub_2023_in_2024.csv")